In [ ]:
%load_ext autoreload
%autoreload 2

import os
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

model_name='Llama2-7b-hf'
#model_name='Llama-3.2-1B'

DTYPE = torch.float16
model = qlib.load_custom_llama(
	model_name=model_name, 
	device_map='cpu', 
	torch_dtype=DTYPE, 
	#use_flash_attn=True
)

In [ ]:
T=256 
L=11
V=2
K=2
tlut_bits=10
decode_mode='quantlut_sym_2d'
incoh_proc_mode='qtip'
viterby_batch_size=3072#1024


wrapper = qlib.HomeQuantWrapper(
    wrap_rule={
        torch.nn.Linear : qlib.TrellisLinear(
			T=T, 
            L=L, 
            V=V, 
            K=K,
			tlut_bits=tlut_bits,
            decode_mode=decode_mode, 
            incoh_proc_mode=incoh_proc_mode, 
            viterby_batch_size=viterby_batch_size
		),
    },
    exceptions = {
        'lm_head' : None,
    }
)
wrapper.wrap_model(model)

model_checkpoints_path = f'/home/msst/repo/Quantization/logs/checkpoints_{model_name}/trellis'
os.makedirs(model_checkpoints_path, exist_ok=True)
torch.save(model, os.path.join(model_checkpoints_path, f'T{T}_L{L}_V{V}_K{K}_{decode_mode}_{incoh_proc_mode}.pth'))


 56%|█████▋    | 144/256 [00:01<00:00, 115.47it/s]


KeyboardInterrupt: 